In [40]:
import warnings 
warnings.filterwarnings("ignore", category = DeprecationWarning)
warnings.filterwarnings("ignore", category = UserWarning)
import MDAnalysis as mda
import nglview as nv
from MDAnalysis.analysis import distances
import numpy as np
import matplotlib.pyplot as plt
import os
import sys 
import glob 
import h5py 
import numpy as np 
from tqdm import tqdm 
from scipy.sparse import csr_matrix

In [41]:
#pdb = mda.Universe('../S1_closed_cleaved_final_min_lipidsw_ions.pdb')
#traj = mda.Universe('../S1_closed_cleaved_final_min_lipidsw_ions.psf', '../pr.*.dcd')

pdb_file = '../S1_closed_cleaved_final_min_lipidsw_ions.pdb' 
dcd_files = sorted(glob.glob('../pr.*.dcd'))

In [42]:
def triu_to_full(cm0):
    num_res = int(np.ceil((len(cm0) * 2) ** 0.5))
    iu1 = np.triu_indices(num_res, 1)

    cm_full = np.zeros((num_res, num_res))
    cm_full[iu1] = cm0
    cm_full.T[iu1] = cm0
    np.fill_diagonal(cm_full, 1)
    return cm_full


In [43]:
mda_traj = mda.Universe(pdb_file, dcd_files)  
protein_ca = mda_traj.select_atoms('protein and name CA') 

In [44]:
len(mda_traj.trajectory[::100])

58

In [56]:
contact_maps = []
for _ in tqdm(mda_traj.trajectory): 
    contact_map = triu_to_full(
        (distances.self_distance_array(protein_ca.positions) < 8.0) * 1.0
        )
    sprs=csr_matrix(contact_map)
    contact_maps.append(sprs) 
    
contact_maps = np.array(contact_maps)


100%|██████████| 5738/5738 [42:52<00:00,  2.34it/s] 


In [60]:
contact_maps.shape
print(contact_maps)


[[<3768x3768 sparse matrix of type '<class 'numpy.float64'>'
	with 39478 stored elements in Compressed Sparse Row format>]
 [<3768x3768 sparse matrix of type '<class 'numpy.float64'>'
	with 39398 stored elements in Compressed Sparse Row format>]
 [<3768x3768 sparse matrix of type '<class 'numpy.float64'>'
	with 39314 stored elements in Compressed Sparse Row format>]
 ...
 [<3768x3768 sparse matrix of type '<class 'numpy.float64'>'
	with 39324 stored elements in Compressed Sparse Row format>]
 [<3768x3768 sparse matrix of type '<class 'numpy.float64'>'
	with 39378 stored elements in Compressed Sparse Row format>]
 [<3768x3768 sparse matrix of type '<class 'numpy.float64'>'
	with 39306 stored elements in Compressed Sparse Row format>]]


In [61]:
# padding if odd dimension occurs in image
pad_f = lambda x: (0, 0) if x % 2 == 0 else (0, 1)
padding_buffer = [(0, 0)]
for x in contact_maps.shape[1:]:
    padding_buffer.append(pad_f(x))
contact_maps = np.pad(contact_maps, padding_buffer, mode='constant')

contact_maps = contact_maps.reshape((contact_maps.shape) + (1,))

In [68]:
# save formulated contact maps to hdf5
cm_h5 = h5py.File('contact_maps_sprs_all.h5', 'w') 
dt = h5py.special_dtype(vlen=np.dtype('float64'))
cm_h5.create_dataset('contact_maps', data=contact_maps, dtype=dt) 
cm_h5.close() 

print('Done')

AttributeError: 'int' object has no attribute 'shape'

In [ ]:
con